In [20]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.formula.api as smf
abspath = '/Users/alexcappadona/Downloads/'
data = pd.read_excel(abspath + 'International Stock Returns and Vaccine News.xlsx', index_col=0)
data.head()

# dummy variabl for Asian

,Index,Symbol,close,return,Announcement,Asian
date,,,,,,
2020-01-22,S&P 50 Asia Index,SPA,5232.18,1.229924,0,1
2020-01-23,S&P 50 Asia Index,SPA,5159.58,-1.387567,0,1
2020-01-24,S&P 50 Asia Index,SPA,5166.30,0.130243,1,1
2020-01-28,S&P 50 Asia Index,SPA,5108.07,-1.127112,0,1
2020-01-29,S&P 50 Asia Index,SPA,5049.61,-1.144464,0,1


In [21]:
# 'return' is a reserved word
data['ret']=data['return']

# creating a variable for announcement*asian dummy
data['AnnAsian']=data['Announcement']*data['Asian']
data['ret_1']=data.groupby('Symbol')['ret'].shift(1) # create a lag for each symbol
# drop missing observations (for day when lag is not available)
data=data.dropna()
data.head()

,Index,Symbol,close,return,Announcement,Asian,ret,AnnAsian,ret_1
date,,,,,,,,,
2020-01-23,S&P 50 Asia Index,SPA,5159.58,-1.387567,0,1,-1.387567,0,1.229924
2020-01-24,S&P 50 Asia Index,SPA,5166.30,0.130243,1,1,0.130243,1,-1.387567
2020-01-28,S&P 50 Asia Index,SPA,5108.07,-1.127112,0,1,-1.127112,0,0.130243
2020-01-29,S&P 50 Asia Index,SPA,5049.61,-1.144464,0,1,-1.144464,0,-1.127112
2020-01-30,S&P 50 Asia Index,SPA,4881.50,-3.329168,0,1,-3.329168,0,-1.144464


In [24]:
spx=data[data['Symbol']=='SPX']
sse=data[data['Symbol']=='SSE']

In [26]:
spx.describe()

,close,return,Announcement,Asian,ret,AnnAsian,ret_1
count,239.000000,239.000000,239.000000,239.0,239.000000,239.0,239.000000
mean,3214.220711,0.076398,0.209205,0.0,0.076398,0.0,0.073824
std,328.026352,2.228770,0.407595,0.0,2.228770,0.0,2.228467
min,2237.400000,-11.984050,0.000000,0.0,-11.984050,0.0,-11.984050
25%,3006.210000,-0.654827,0.000000,0.0,-0.654827,0.0,-0.654827
50%,3276.020000,0.239693,0.000000,0.0,0.239693,0.0,0.235373
75%,3429.120000,1.049207,0.000000,0.0,1.049207,0.0,1.049207
max,3756.070000,9.382766,1.000000,0.0,9.382766,0.0,9.382766


In [39]:
formula = 'ret ~ ret_1 + Announcement' # Regression for S&P 500 index
results = smf.ols(formula, spx).fit(cov_type='HC0')
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                    ret   R-squared:                       0.162
Model:                            OLS   Adj. R-squared:                  0.155
Method:                 Least Squares   F-statistic:                     11.37
Date:                Wed, 27 Oct 2021   Prob (F-statistic):           1.93e-05
Time:                        14:24:57   Log-Likelihood:                -509.05
No. Observations:                 239   AIC:                             1024.
Df Residuals:                     236   BIC:                             1035.
Df Model:                           2                                         
Covariance Type:                  HC0                                         
                   coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept       -0.1212      0.155     -0.780   

In [40]:
formula = 'ret ~ ret_1 + Announcement' # Regression for Shanghai Index
results = smf.ols(formula, sse).fit(cov_type='HC0')
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                    ret   R-squared:                       0.005
Model:                            OLS   Adj. R-squared:                 -0.004
Method:                 Least Squares   F-statistic:                    0.2913
Date:                Wed, 27 Oct 2021   Prob (F-statistic):              0.748
Time:                        14:25:36   Log-Likelihood:                -388.91
No. Observations:                 228   AIC:                             783.8
Df Residuals:                     225   BIC:                             794.1
Df Model:                           2                                         
Covariance Type:                  HC0                                         
                   coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept        0.0452      0.107      0.423   

In [41]:
# Chienese markets did not react as much to vaccine announcement news
# They have their own vaccines & their econ was not as effected by covid

In [42]:
# Regression with interaction for Asia

formula = 'ret ~ ret_1 + Announcement + Asian + AnnAsian' # Regression for Shanghai Index
results = smf.ols(formula, data).fit(cov_type='HC0')
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                    ret   R-squared:                       0.031
Model:                            OLS   Adj. R-squared:                  0.029
Method:                 Least Squares   F-statistic:                     10.12
Date:                Wed, 27 Oct 2021   Prob (F-statistic):           4.20e-08
Time:                        14:29:10   Log-Likelihood:                -3796.7
No. Observations:                1898   AIC:                             7603.
Df Residuals:                    1893   BIC:                             7631.
Df Model:                           4                                         
Covariance Type:                  HC0                                         
                   coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept       -0.1721      0.071     -2.420   

In [43]:
# Vaccine announcement matters when the country we are looking at is 0
# 'Announcement' = average impact on announcement
# .86 - .55 == total impact of announcement on asian markets

In [46]:
# Regression by group

grp = data.groupby('Symbol')
# loop: regression for each symbol
for Symbol, group in grp:
    print(Symbol)
    formula = 'ret ~ ret_1 + Announcement'
    results = smf.ols(formula, group).fit(cov_type='HC0')
    print(results.summary())

DAX
                            OLS Regression Results                            
Dep. Variable:                    ret   R-squared:                       0.029
Model:                            OLS   Adj. R-squared:                  0.021
Method:                 Least Squares   F-statistic:                     3.168
Date:                Wed, 27 Oct 2021   Prob (F-statistic):             0.0439
Time:                        14:41:25   Log-Likelihood:                -516.23
No. Observations:                 239   AIC:                             1038.
Df Residuals:                     236   BIC:                             1049.
Df Model:                           2                                         
Covariance Type:                  HC0                                         
                   coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept       -0.1645      0.149     -1.10

In [47]:
# get mean for each variable grouped by symbol
data.groupby('Symbol').mean()

,close,return,Announcement,Asian,ret,AnnAsian,ret_1
Symbol,,,,,,,
DAX,12271.832008,0.029139,0.217573,0.0,0.029139,0.000000,0.029196
FTSE,6192.480837,-0.048230,0.217573,0.0,-0.048230,0.000000,-0.044279
HSIX,25092.524678,-0.005962,0.223176,1.0,-0.005962,0.223176,-0.001835
KS200,296.182747,0.120514,0.218884,1.0,0.120514,0.218884,0.117126
SPA,5249.568156,0.114157,0.213115,1.0,0.114157,0.213115,0.117840
SPX,3214.220711,0.076398,0.209205,0.0,0.076398,0.000000,0.073824
SSE,3131.245307,0.064427,0.223684,1.0,0.064427,0.223684,0.058135
SXXP,366.124568,-0.007734,0.213992,0.0,-0.007734,0.000000,-0.006810


In [49]:
# get std dev for each variable grouped by symbol
data.groupby('Symbol').mean()

,close,return,Announcement,Asian,ret,AnnAsian,ret_1
Symbol,,,,,,,
DAX,12271.832008,0.029139,0.217573,0.0,0.029139,0.000000,0.029196
FTSE,6192.480837,-0.048230,0.217573,0.0,-0.048230,0.000000,-0.044279
HSIX,25092.524678,-0.005962,0.223176,1.0,-0.005962,0.223176,-0.001835
KS200,296.182747,0.120514,0.218884,1.0,0.120514,0.218884,0.117126
SPA,5249.568156,0.114157,0.213115,1.0,0.114157,0.213115,0.117840
SPX,3214.220711,0.076398,0.209205,0.0,0.076398,0.000000,0.073824
SSE,3131.245307,0.064427,0.223684,1.0,0.064427,0.223684,0.058135
SXXP,366.124568,-0.007734,0.213992,0.0,-0.007734,0.000000,-0.006810
